# VES equivalence

In this exercise you will investigate VES through forward modelling. We will be using the SimPEG code released by UBC and others https://simpeg.xyz/. Installation is simple:

In [ ]:
! pip install simpeg matplotlib 
import os
import numpy as np
import matplotlib.pyplot as plt

from discretize import TensorMesh

from SimPEG import maps
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.electromagnetics.static.utils.static_utils import plot_layer
plt.style.use('ggplot')
save_file = False

# Survey configuration
We begin by setting up a Wener sounding array, this code is borrowed from SimPEG examples 


In [ ]:
#####################################################################
# Create Survey
# -------------
#
# Here we demonstrate a general way to define sources and receivers.
# For pole and dipole sources, we must define the A or AB electrode locations,
# respectively. For the pole and dipole receivers, we must define the M or
# MN electrode locations, respectively.
#

a_min = 20.0
a_max = 100.0
n_stations = 25

# Define the 'a' spacing for Wenner array measurements for each reading
electrode_separations = np.linspace(a_min, a_max, n_stations)

source_list = []  # create empty array for sources to live

for ii in range(0, len(electrode_separations)):

    # Extract separation parameter for sources and receivers
    a = electrode_separations[ii]

    # AB electrode locations for source. Each is a (1, 3) numpy array
    A_location = np.r_[-1.5 * a, 0.0, 0.0]
    B_location = np.r_[1.5 * a, 0.0, 0.0]

    # MN electrode locations for receivers. Each is an (N, 3) numpy array
    M_location = np.r_[-0.5 * a, 0.0, 0.0]
    N_location = np.r_[0.5 * a, 0.0, 0.0]

    # Create receivers list. Define as pole or dipole.
    receiver_list = dc.receivers.Dipole(M_location, N_location)
    receiver_list = [receiver_list]

    # Define the source properties and associated receivers
    source_list.append(dc.sources.Dipole(receiver_list, A_location, B_location))

# Define survey
survey = dc.Survey(source_list)

# Define a 1D conductivity model

In [ ]:
###############################################
# Defining a 1D Layered Earth Model
# ---------------------------------
#
# Here, we define the layer thicknesses and electrical resistivities for our
# 1D simulation. If we have N layers, we define N electrical resistivity
# values and N-1 layer thicknesses. The lowest layer is assumed to extend to
# infinity.
#

# Define layer thicknesses.
layer_thicknesses = np.r_[50.0, 20.0]

# Define layer resistivities.
model = np.r_[2e3, 1e1, 2e3]

print("Layer 2 conductance (h / rho) (model)", layer_thicknesses[1]/model[1] )

# Define mapping from model to 1D layers.
model_map = maps.IdentityMap(nP=len(model))

###############################################################
# Plot Resistivity Model
# ----------------------

# Define a 1D mesh for plotting. Provide a maximum depth for the plot.
max_depth = 200
mesh = TensorMesh([np.r_[layer_thicknesses, max_depth - layer_thicknesses.sum()]])

# Plot the 1D model
plot_layer(model_map * model, mesh, color='red')

# Perform forward modelling
We use SimPEG to calculate the forward response

In [ ]:
#######################################################################
# Define the Forward Simulation and Predict DC Resistivity Data
# -------------------------------------------------------------
#
# Here we predict DC resistivity data. If the keyword argument *rhoMap* is
# defined, the simulation will expect a resistivity model. If the keyword
# argument *sigmaMap* is defined, the simulation will expect a conductivity model.
#

simulation = dc.simulation_1d.Simulation1DLayers(
    survey=survey,
    rhoMap=model_map,
    thicknesses=layer_thicknesses,
    data_type="apparent_resistivity",
)

# Predict data for a given model
dpred = simulation.dpred(model)

# Plot apparent resistivities on sounding curve
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_axes([0.1, 0.1, 0.75, 0.85])
ax1.semilogy(1.5 * electrode_separations, dpred, "red")
ax1.set_xlabel("AB/2 (m)")
ax1.set_ylabel("Apparent Resistivity ($\Omega m$)")
plt.show()



# Define a second conductivity model

In [ ]:
###############################################
# Defining a 1D Layered Earth Model
# ---------------------------------
#
# Here, we define the layer thicknesses and electrical resistivities for our
# 1D simulation. If we have N layers, we define N electrical resistivity
# values and N-1 layer thicknesses. The lowest layer is assumed to extend to
# infinity.
#

# Define layer thicknesses.
layer_thicknesses2 = np.r_[50.0, 10.]

# Define layer resistivities.
model2 = np.r_[2e3, 5, 2e3]

print("Layer 2 conductance (model2)", layer_thicknesses2[1]/model2[1] )

# Define mapping from model to 1D layers.
model_map2 = maps.IdentityMap(nP=len(model2))

###############################################################
# Plot Resistivity Model
# ----------------------

# Define a 1D mesh for plotting. Provide a maximum depth for the plot.
max_depth = 200
mesh2 = TensorMesh([np.r_[layer_thicknesses2, max_depth - layer_thicknesses2.sum()]])

# Plot the 1D model
plot_layer(model_map * model, mesh, color='red', label='model1', linewidth=3)
plot_layer(model_map2 * model2, mesh2, label='model2')
plt.legend()


In [ ]:
#######################################################################
# Define the Forward Simulation and Predict DC Resistivity Data
# -------------------------------------------------------------
#
# Here we predict DC resistivity data. If the keyword argument *rhoMap* is
# defined, the simulation will expect a resistivity model. If the keyword
# argument *sigmaMap* is defined, the simulation will expect a conductivity model.
#

# Predict data for a given model
dpred2 = simulation.dpred(model2)

# Plot apparent resistivities on sounding curve
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_axes([0.1, 0.1, 0.75, 0.85])
ax1.semilogy(1.5 * electrode_separations, dpred2, "b")
ax1.set_xlabel("AB/2 (m)")
ax1.set_ylabel("Apparent Resistivity ($\Omega m$)")
plt.show()

# Plot the difference

In [ ]:
# Plot apparent resistivities on sounding curve
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_axes([0.1, 0.1, 0.75, 0.85])
ax1.semilogy(1.5 * electrode_separations, dpred, "red", label='model1 sounding')
ax1.semilogy(1.5 * electrode_separations, dpred2, "black", label='model2 sounding')
#ax3 = plt.twinx()
ax1.semilogy(1.5 * electrode_separations, np.abs(dpred-dpred2), '--', color="black", label='difference')
ax1.set_xlabel("AB/2 (m)")
ax1.set_ylabel("Apparent Resistivity ($\Omega m$)")
plt.legend()
plt.show()

# Let's save these sounding curves...

In [ ]:
if True:

    dir_path = os.getcwd() #.split(os.path.sep)
    #print("here", dir_path)
    dir_path += os.path.sep + "equivalence"
    #dir_path = os.path.sep.join(dir_path) + os.path.sep

    print(os.getcwd() )
    print( os.getcwd().split(os.path.sep) )
    print("Saving files in ", dir_path)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    dir_path += os.path.sep 
    
    noise = 0.025 * dpred * np.random.rand(len(dpred))

    data_array = np.c_[
        survey.locations_a,
        survey.locations_b,
        survey.locations_m,
        survey.locations_n,
        dpred + noise,
    ]

    fname = dir_path + "model1.dobs"
    
    np.savetxt(fname, data_array, fmt="%.4e")

    fname = dir_path + "true_model1.txt"
    np.savetxt(fname, model, fmt="%.4e")

    fname = dir_path + "layers1.txt"
    np.savetxt(fname, mesh.hx, fmt="%d")
    
    # second model 
    
    noise2 = 0.025 * dpred2 * np.random.rand(len(dpred))
    data_array2 = np.c_[
        survey.locations_a,
        survey.locations_b,
        survey.locations_m,
        survey.locations_n,
        dpred2 + noise2,
    ]
    
    # model 2
    fname = dir_path + "model2.dobs"
    np.savetxt(fname, data_array2, fmt="%.4e")

    fname = dir_path + "true_model2.txt"
    np.savetxt(fname, model2, fmt="%.4e")

    fname = dir_path + "layers2.txt"
    np.savetxt(fname, mesh2.hx, fmt="%d")

# Questions:
 * Why is this happening? 
 * What impact do you think this will have on inversion? 
 * The longitudinal conductance is constant for the above, why would this be an important factor? 
 
 
# Assigment
Modify this example for the case of an 20 metre thick insulator (500 Ohm m)between two conductive layers (20 Ohm m). 
 * Can you find an equivalent layer in this situation? 
 * What factor has to be constant?
